# FORK 1
## parallel_in_python3_fork1.ipynb

In [4]:
import sys
import os
from glob import glob
import numpy as np
import pandas as pd
from cluster_helper.cluster import cluster_view
sys.path.append('/home1/dscho/code/general')
import data_io as dio

In [51]:
def calc_power_by_pl_unit_to_region_parallel(fname):
    import os
    import sys
    sys.path.append('/home1/dscho/code/projects/unit_activity_and_hpc_theta')
    import phase_locking
    sys.path.append('/home1/dscho/code/general')
    import data_io as dio
    
    try:
        info = dio.open_pickle(fname)
        _ = phase_locking.calc_power_by_pl_unit_to_region(info,
                                                          sleep_max=1200)
        return None
    except:
        err = sys.exc_info()
        subj_sess = info.subj_sess
        unit = info.unit
        lfp_roi = info.lfp_hemroi
        errf = '/home1/dscho/logs/TryExceptError-calc_power_by_pl_unit_to_region_parallel-{}-unit_{}-lfp_{}'.format(subj_sess, unit, lfp_roi)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err)+'\n')
        return None
    
def calc_power_across_spikes_unit_to_region_parallel(fname):
    import os
    import sys
    sys.path.append('/home1/dscho/code/projects/unit_activity_and_hpc_theta')
    import phase_locking
    sys.path.append('/home1/dscho/code/general')
    import data_io as dio
    
    try:
        info = dio.open_pickle(fname)
        _ = phase_locking.calc_power_across_spikes_unit_to_region(info,
                                                                  sleep_max=300)
        return None
    except:
        err = sys.exc_info()
        subj_sess = info.subj_sess
        unit = info.unit
        lfp_roi = info.lfp_hemroi
        errf = '/home1/dscho/logs/TryExceptError-calc_power_across_spikes_unit_to_region_parallel-{}-unit_{}-lfp_{}'.format(subj_sess, unit, lfp_roi)
        os.system('touch {}'.format(errf))
        with open(errf, 'w') as f:
            f.write(str(err)+'\n')
        return None

In [21]:
sessions1 = ['U387_ses3', 'U369_env1', 'U372_ses2', 'U390_ses3', 
             'U369_env2', 'U373_ses1', 'U387_ses1', 'U380_ses2',
             'U393_ses2', 'U395_ses2', 'U384_ses3', 'U385_ses4', 
             'U371_env2', 'U371_env3']

sessions2 = ['U379_ses1', 'U380_ses1a', 'U381_ses1', 'U390_ses1',
             'U394_ses3', 'U395_ses3', 'U370_env1', 'U371_env1',
             'U372_ses1', 'U373_ses2', 'U373_ses3', 'U374_ses1', 
             'U374_ses2', 'U380_ses3', 'U381_ses2', 'U381_ses3', 
             'U384_ses1', 'U384_ses2', 'U387_ses2', 'U390_ses2', 
             'U390_ses4', 'U392_ses1', 'U394_ses1', 'U395_ses1', 
             'U396_ses2', 'U396_ses3', 'U385_ses1', 'U385_ses2', 
             'U385_ses3']

sessions_nohpc = ['U386_ses1', 'U386_ses2'] 

sessions = sessions1 + sessions2 #+ sessions_nohpc

print(len(sessions))

45


In [17]:
# Who has power files?
files = glob('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/wavelet/power/*')
subjs = list(np.unique([os.path.basename(x)[12:21] for x in files]))
np.unique([os.path.basename(x)[12:21] for x in files], return_counts=True)

(array(['U369_env1', 'U369_env2', 'U370_env1', 'U371_env1', 'U371_env2',
        'U371_env3', 'U372_ses1', 'U372_ses2', 'U373_ses1', 'U373_ses2',
        'U373_ses3', 'U374_ses1', 'U374_ses2', 'U379_ses1', 'U380_ses1',
        'U380_ses2', 'U380_ses3', 'U381_ses1', 'U381_ses2', 'U381_ses3',
        'U384_ses1', 'U384_ses2', 'U384_ses3', 'U385_ses1', 'U385_ses2',
        'U385_ses3', 'U385_ses4', 'U387_ses1', 'U387_ses2', 'U387_ses3',
        'U390_ses1', 'U390_ses2', 'U390_ses3', 'U390_ses4', 'U392_ses1',
        'U393_ses2', 'U394_ses1', 'U394_ses3', 'U395_ses1', 'U395_ses2',
        'U395_ses3', 'U396_ses2', 'U396_ses3'], dtype='<U9'),
 array([256, 256, 256, 384, 384, 384, 256, 256, 128, 128, 128, 128, 128,
        256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256,
        256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256,
        256, 256, 384, 384]))

In [43]:
files = glob('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_for_power/*pkl')
files_to_process = []
for f in files:
    fname = os.path.basename(f).split('-')
    subj_sess = fname[1]
    unit = fname[2][5:]
    lfp_roi = fname[3][4:]
    if not os.path.exists(os.path.join('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/power_by_pl_fr',
                                       'power_by_phase_locking-{}-unit_{}-lfp_{}-2000Hz-notch60_120Hz5cycles-16log10freqs_0.5_to_90.5Hz.pkl'
                                       .format(subj_sess, unit, lfp_roi))):
        files_to_process.append(f)
print('{}/{} files to process'.format(len(files_to_process), len(files)))

470/470 files to process


In [45]:
# Parallel processing
print('Running code for {} jobs.\n'.format(len(files_to_process)))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=min(200, len(files_to_process)), cores_per_job=1) as view: 
    output = view.map(calc_power_by_pl_unit_to_region_parallel, files_to_process)

In [47]:
files = glob('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_for_power/*pkl')
files_to_process = []
for f in files:
    fname = os.path.basename(f).split('-')
    subj_sess = fname[1]
    unit = fname[2][5:]
    lfp_roi = fname[3][4:]
    if not os.path.exists(os.path.join('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/power_by_pl_fr/mean_power_across_spikes',
                                       'mean_Z_log_power_across_spikes-{}-unit_{}-lfp_{}-2000Hz-notch60_120Hz5cycles-16log10freqs_0.5_to_90.5Hz.pkl'
                                       .format(subj_sess, unit, lfp_roi))):
        files_to_process.append(f)
print('{}/{} files to process'.format(len(files_to_process), len(files)))

2309/2310 files to process


In [52]:
# Parallel processing
print('Running code for {} jobs.\n'.format(len(files_to_process)))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=min(200, len(files_to_process)), cores_per_job=1) as view: 
    output = view.map(calc_power_across_spikes_unit_to_region_parallel, files_to_process)

Running code for 2309 jobs.

79 Engines running
Sending a shutdown signal to the controller and engines.


OSError: [Errno 16] Device or resource busy: '.nfs0000004500fefcad000001e3'